In [2]:
import argparse
import json
import os
import random

import numpy as np

import ray
from ray.tune import Trainable, run, sample_from
from ray.tune.schedulers import AsyncHyperBandScheduler





In [3]:
ahb = AsyncHyperBandScheduler(time_attr='training_iteration', \
                              metric='episode_reward_mean', mode='max',\
                             grace_period=5,max_t=100)

In [5]:
class MyTrainableClass(Trainable):
    
    def _setup(self,config):
        self.timestep = 0
        
    def _train(self):
        self.timestep = 1
        v = np.tanh(float(self.timestep)/ self.config.get('width',1))
        v *= self.config.get('height',1)
        return v
    
    def _save(self,checkpoint_dir):
        path = os.path.join(checkpoint_dir,'checkpoint')
        with open(path,'w') as f:
            f.write(json.dumps({'timestep':self.timestep}))
        return path
    
    def _restore(self,checkpoint_path):
        with open(checkpoint_path) as f:
            self.timestep = json.loads(f.read())['timestep']
            
            

In [12]:
run(MyTrainableClass, name='asynchyperband_test',scheduler='ahb',\
   stop={'trainable_iteration': 1},\
   num_samples=20, resources_per_trial={'cpu':1,'gpu':0},\
   config={'width':sample_from(lambda spec: 10 + int(90*random.random())),\
          'height':sample_from(lambda spec: int(100*random.random()))},\
   verbose=0 )

AttributeError: 'str' object has no attribute 'on_trial_add'